# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# print current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# loop each file in the path and make a list of these files
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# prints the number of rows
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

# connects to local machine to a Cassandra instance 
cluster = Cluster()
session = cluster.connect()

#### Create and Set Keyspace

In [6]:
# Create KEYSPACE songs_analysis
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS songs_analysis 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

# Set KEYSPACE to the songs_analysis
session.set_keyspace('songs_analysis')

### Query 1

In [7]:
# The query returns the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS artist_song_and_songLength"
query = query + "(sessionId int, \
                    itemInSession int, \
                    artist text, \
                    song text, \
                    length float, \
                    PRIMARY KEY(sessionId,itemInSession))"
session.execute(query)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_and_songLength (sessionId,itemInSession,artist,song,length)"
        query = query + "VALUES (%s, %s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0], line[9],float(line[5])))                    

In [8]:
# test the table
query="SELECT * FROM artist_song_and_songLength LIMIT 5"
results=session.execute(query)
for row in results:
    print(row)

Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=191.08526611328125, song='The Calculation (Album Version)')
Row(sessionid=23, iteminsession=1, artist='Octopus Project', length=250.95791625976562, song='All Of The Champs That Ever Lived')
Row(sessionid=23, iteminsession=2, artist='Tegan And Sara', length=180.06158447265625, song='So Jealous')
Row(sessionid=23, iteminsession=3, artist='Dragonette', length=153.39056396484375, song='Okay Dolores')
Row(sessionid=23, iteminsession=4, artist='Lil Wayne / Eminem', length=229.58975219726562, song='Drop The World')


In [9]:
# Result of Query 1
query1="SELECT artist,song,length FROM artist_song_and_songLength WHERE sessionId=338 AND itemInSession=4"
results=session.execute(query1)
for row in results:
    print(row.artist,row.song,row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2

In [10]:
## The query returns only the name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS artist_song_userName"
query = query + "(userid int, \
                    sessionid int, \
                    itemInSession int,\
                    artist text,\
                    song text,\
                    firstName text,\
                    lastName text,\
                    PRIMARY KEY((userId,sessionId),itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC)"
session.execute(query)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_userName (userId,sessionId,itemInSession,artist,song,firstName,lastName)"
        query = query + "VALUES (%s, %s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0], line[9],line[1],line[4]))    

                    

In [11]:
# Test the table
query="SELECT * FROM artist_song_userName LIMIT 5"
results=session.execute(query)
for row in results:
    print(row)

Row(userid=58, sessionid=768, iteminsession=0, artist='System of a Down', firstname='Emily', lastname='Benson', song='Sad Statue')
Row(userid=58, sessionid=768, iteminsession=1, artist='Ghostland Observatory', firstname='Emily', lastname='Benson', song='Stranger Lover')
Row(userid=58, sessionid=768, iteminsession=2, artist='Evergreen Terrace', firstname='Emily', lastname='Benson', song='Zero')
Row(userid=85, sessionid=776, iteminsession=2, artist='Deftones', firstname='Kinsley', lastname='Young', song='Head Up (LP Version)')
Row(userid=85, sessionid=776, iteminsession=3, artist='The Notorious B.I.G.', firstname='Kinsley', lastname='Young', song='Playa Hater (Amended Version)')


In [12]:
# Result of Query 2
query2="SELECT artist,song,firstname,lastname FROM artist_song_userName WHERE userId=10 AND sessionid = 182" 
results=session.execute(query2)
for row in results:
    print(row.artist,row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3

In [13]:
## # The query returns every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS userName"
query = query + "(song text,\
                    firstName text,\
                    lastName text,\
                    PRIMARY KEY(song,firstName,lastName))"
session.execute(query)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO userName (song,firstName,lastName)"
        query = query + "VALUES (%s, %s,%s)"
        session.execute(query, (line[9],line[1],line[4]))

                    

In [14]:
# Test the table
query="SELECT * FROM userName LIMIT 5"
results=session.execute(query)
for row in results:
    print(row)

Row(song="Wonder What's Next", firstname='Chloe', lastname='Cuevas')
Row(song="In The Dragon's Den", firstname='Chloe', lastname='Cuevas')
Row(song='Too Tough (1994 Digital Remaster)', firstname='Aleena', lastname='Kirby')
Row(song='Rio De Janeiro Blue (Album Version)', firstname='Chloe', lastname='Cuevas')
Row(song='My Place', firstname='Jacob', lastname='Klein')


In [15]:
query3="SELECT firstname,lastname FROM userName WHERE song='All Hands Against His Own'"
results=session.execute(query3)
for row in results:
    print(row.firstname,row.lastname)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [16]:
## TO-DO: Drop the table before closing out the sessions
session.execute( "DROP TABLE artist_song_and_songLength")
session.execute( "DROP TABLE artist_song_userName")
session.execute( "DROP TABLE userName")

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()